In [1]:
import requests 
import yaml
import json
import datetime as dt
import time
import os
import sys
import random
from azure.storage.blob import BlobServiceClient, BlobType
from email_validator import validate_email, EmailNotValidError
from itertools import chain

# pathing 
PARENT_DIR = os.path.abspath(os.path.pardir)
module_path = os.path.abspath(os.path.join(PARENT_DIR)) # for other functions
if module_path not in sys.path:
  sys.path.append(module_path)
from funcs.utils import log_azure, request, load_json, clean_field_text, load_config, get_email, send_email, create_hyperlink
from funcs.funcs import combine_qa_keys, get_sm_survey_responses, process_sm_responses, post_cos

DATA_DIR = os.path.join(PARENT_DIR, "data")

# creds
with open(os.path.join(PARENT_DIR, "creds", "api-key.yaml"), "r") as file:
    data = yaml.full_load(file)

# SurveyMonkey Survey
SM_DATA = data['sm']['real']
# CareerOneStop Survey 
COS_DATA = data['cos']


---

##### **Main Functions** 

* `get_qa_key()` - GET (or load cached copy) of question/answer key from SM or COS 

<br>

* `combine_qa_keys()` - Combine the SM and COS question/answer keys into one combined key/translation map between the APIs
    - Generates a refreshed map if a change is detected in the SM survey or the COS survey. 
        - The COS survey should not change at all. 
        - The survey monkey questions which match to the COS survey questions should not change (they are intended to just be a port).

<br>

* `get_sm_responses()` - GET SM survey responses 

<br>

* `process_sm_responses()` - filter and process new SM survey responses from get_sm_responses()
    - Checks against DB for already processed responses 
    - Checks for unexpected question ids vs. the combined Q/A key.
        - Attempts to refresh the combined Q/A key if any unexpected question ids are found.
    - Adds matching information from the combined Q/A key to the survey responses   
    - Loads new responses into database (into 'processing' table) until they are finished

<br>

* `post_cos()` 
    - Creates COS JSON request objects from each SM survey response object 
        - If a SM survey response is missing an answer for a skills-survey question, it fills the corresponding question in the COS object with an answer of "Beginner"
    -  POSTS each request object to the COS Skills Matcher API 
    - Stores the COS response alongside the original SM survey response, updating the database 
    
<br>

* `send_email()` - Send email if respondent provided valid email address.   

In [10]:
sm_survey_responses['data'][0]['pages'][3]['questions']

[{'id': '156451000', 'answers': [{'choice_id': '1149785289'}]},
 {'id': '156451001',
  'answers': [{'tag_data': [], 'text': 'Data Engineering Fellow'}]},
 {'id': '156451002', 'answers': [{'choice_id': '1149785293'}]}]

In [2]:
## DEMO 10/16
# sm_survey_responses = get_sm_survey_responses()
processed_sm_responses = process_sm_responses(sm_survey_responses)
# processed_sm_cos_responses = post_cos(processed_sm_responses)
# send_email(cos_recommendations[0])

processed_sm_responses[0]['questions']

INFO: GET {'https://api.surveymonkey.com/v3/surveys/409913146/responses/bulk'} -- {200} -- 1.65 -- {datetime.datetime(2023, 10, 18, 11, 2, 26, 660930)}
['156451072', '156451074', '156451075', '156450996', '156450999', '156451000', '156451001', '156451002', '156451004', '156451007', '156451008', '156451070', '156451019', '156451020', '156451021', '156451022', '156451023', '156451024', '156451026', '156451027', '156451028', '156451029', '156451030', '156451032', '156451033', '156451034', '156451035', '156451038', '156451039', '156451040', '156451041', '156451042', '156451043', '156451044', '156451045', '156451046', '156451047', '156451048', '156451049', '156451050', '156451051', '156451052', '156451053', '156451054', '156451055', '156451056', '156451057', '156451059', '156451060', '156451061', '156451062', '156451063', '156451064', '156451065', '156451066', '156451067', '156451068']
{'sm': '156451031', 'cos': '1.A.3.c.3'}
{'question_id': {'sm': '156451031', 'cos': '1.A.3.c.3'}, 'page_num

/Users/alexsocarras/tech_impact/career-onestop/funcs/utils.py:100: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


[{'question_id': {'sm': '156451072'},
  'page_number': 1,
  'question_number': {'sm': 1},
  'question_family': 'single_choice',
  'question_text': {'sm': 'How did you learn about the survey?'},
  'question_type': 'non-skills-matcher',
  'answers': [{'id': {'sm': '1149785635'},
    'text': {'sm': 'I am working on it (Alex again!)'}}]},
 {'question_id': {'sm': '156451074'},
  'page_number': 1,
  'question_number': {'sm': 2},
  'question_family': 'single_choice',
  'question_text': {'sm': 'Are you taking this survey online or in person?'},
  'question_type': 'non-skills-matcher',
  'answers': [{'id': {'sm': '1149785637'}, 'text': {'sm': 'Online'}}]},
 {'question_id': {'sm': '156451075'},
  'page_number': 1,
  'question_number': {'sm': 5},
  'question_family': 'single_choice',
  'question_text': {'sm': 'Which of the following best describes you?'},
  'question_type': 'non-skills-matcher',
  'answers': [{'id': {'sm': '1149785643'},
    'text': {'sm': 'Multiracial or Biracial'}}]},
 {'questi

In [21]:
sm_survey_responses['data'][0]

{'id': '114439142203',
 'recipient_id': '',
 'collection_mode': 'default',
 'response_status': 'completed',
 'custom_value': '',
 'first_name': '',
 'last_name': '',
 'email_address': '',
 'ip_address': '96.64.76.209',
 'logic_path': {},
 'metadata': {'contact': {}},
 'page_path': [],
 'collector_id': '428261927',
 'survey_id': '409913146',
 'custom_variables': {},
 'edit_url': 'https://www.surveymonkey.com/r/?sm=NGKxWpKHiSQN3_2Ba3z5zRBknb2iGX_2BF_2BYcuqJSP8ArA7hYFtVBm24yaFqhfYSgwOY',
 'analyze_url': 'https://www.surveymonkey.com/analyze/browse/K2eZS103V3YJHnEu7GMcQyxP7EvTeHqb_2FqFbm1fwE2E_3D?respondent_id=114439142203',
 'total_time': 296,
 'date_modified': '2023-10-17T17:55:14+00:00',
 'date_created': '2023-10-17T17:50:17+00:00',
 'href': 'https://api.surveymonkey.com/v3/surveys/409913146/responses/114439142203',
 'pages': [{'id': '46616907',
   'questions': [{'id': '156451072',
     'answers': [{'other_id': '1149785635',
       'text': 'I am working on it (Alex again!)'}]},
    {'id

In [31]:
combined_map = combine_qa_keys()
skills_matcher_ids = set(combined_map['skills-matcher'].keys())
non_skills_matcher_ids = set(combined_map['non-skills-matcher'].keys())

processed_responses = []
for resp in sm_survey_responses['data']: 
    if resp['id'] not in []: # and has_valid_email(resp check_deliverability=False) 
        # TO-DO: Do the email validation in the actual email sending function 
        resp_dict = {
        'response_id':resp['id'],
        'collector_id':resp['collector_id'], 
        'questions':[] 
        }

        ## Add questions information
        for p in resp['pages']:
            for q_resp in p['questions']:
                # e.g. q_resp = {'id': '156451072', 'answers': [{'other_id': '1149785635', 'text': 'Online'}]}
                # e.g. q_resp = {'id': '156451075', 'answers': [{'choice_id': '1149785640'}]}
                # e.g. q_resp = {'id': '143922396', 'answers': [{'tag_data': [], 'text': '19977'}]}

                # Match given question (q_resp) to question object in combined answer key (q_map)
                question_type = 'non-skills-matcher' if q_resp['id'] in non_skills_matcher_ids else 'skills-matcher'
                q_map = combined_map[question_type][q_resp['id']] # match based on question type and sm question id 

                if q_map['answers'] is not None: # if the answer key has answer choices listed for the question
                        
                    q_map_answer_key = {a['id']['sm']:a for a in q_map['answers']}
                    
                    answers = [q_map_answer_key[a['choice_id']] 
                                for a in q_resp['answers'] if 'choice_id' in a.keys()] \
                            + [{'id':{'sm':a['other_id']}, 'text':{'sm':a['text']}} 
                                for a in q_resp['answers'] if 'other_id' in a.keys()] 
                else:
                    answers = q_resp['answers']
                    
                resp_dict['questions'].append({'question_id':q_map['question_id'], 
                                            'page_number':q_map['page_number'],
                                            'question_number':q_map['question_number'], 
                                            'question_family': q_map['question_family'],
                                            'question_text':q_map['question_text'],
                                            'question_type':question_type, 
                                            'answers':answers})
                    
        current_resp_question_ids = [q['question_id']['sm'] for q in resp_dict['questions']]

        for q_map in list(combined_map['non-skills-matcher'].values()) + list(combined_map['skills-matcher'].values()):
            if q_map['question_id']['sm'] not in current_resp_question_ids:

                auto_fill_answer = [q_map['answers'][0]] if q_map['question_type'] == 'skills-matcher' else None # Auto fill with the beginner answer

                fill_dict = {
                    'question_id':q_map['question_id'], 
                    'page_number':q_map['page_number'],
                    'question_number':q_map['question_number'], 
                    'question_family': q_map['question_family'],
                    'question_text':q_map['question_text'],
                    'question_type':q_map['question_type'], 

                    # Addressing omitted question 
                    'answers':auto_fill_answer,
                    'omitted':True
                }

                resp_dict['questions'].append(fill_dict)
        
                resp_dict['questions'] = sorted(resp_dict['questions'], key=lambda q:int(q['question_number']['sm']))

    processed_responses.append(resp_dict)

processed_responses

[{'response_id': '114439142203',
  'collector_id': '428261927',
  'questions': [{'question_id': {'sm': '156451072'},
    'page_number': 1,
    'question_number': {'sm': 1},
    'question_family': 'single_choice',
    'question_text': {'sm': 'How did you learn about the survey?'},
    'question_type': 'non-skills-matcher',
    'answers': [{'id': {'sm': '1149785635'},
      'text': {'sm': 'I am working on it (Alex again!)'}}]},
   {'question_id': {'sm': '156451074'},
    'page_number': 1,
    'question_number': {'sm': 2},
    'question_family': 'single_choice',
    'question_text': {'sm': 'Are you taking this survey online or in person?'},
    'question_type': 'non-skills-matcher',
    'answers': [{'id': {'sm': '1149785637'}, 'text': {'sm': 'Online'}}]},
   {'question_id': {'sm': '156450994'},
    'page_number': 1,
    'question_number': {'sm': 3},
    'question_family': 'open_ended',
    'question_text': {'sm': 'What zip code do you currently live in?'},
    'question_type': 'skills-mat

In [3]:
from tabulate import tabulate

with open('../creds/api-key.yaml', 'r') as file: 
    data = yaml.full_load(file)['elastic-email']['shared-account']

APP_PASSWORD = data['app-password']
API_KEY = data['ee-api-key']
SENDER_EMAIL = data['sender-email']
RECEIVER_EMAIL = get_email(cos_recommendations)

# general email settings 
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_SUBJECT = f'Your survey result for {dt.datetime.now().strftime("%B %d, %Y")}'

# Extract and format data
cos_response = cos_recommendations['cos_response']
rec_list = cos_response['SKARankList']
rename_keys_map = {'Rank': 'Your Match Rank', 
                'OccupationTitle': 'Occupation Title', 
                'AnnualWages': 'Average Wages (Annual)', 
                'TypicalEducation': 'Typical Education'
                }
for rec in rec_list:
    # Set occupation title to hyperlink
    rec['OccupationTitle'] = create_hyperlink(rec)
    # Drop redundant field (used to create the hyperlink)
    rec.pop('OnetCode')
    # Format wages 
    rec['AnnualWages'] = f"${rec['AnnualWages']:,.0f}"
    # Rename columns
    for k,v in rename_keys_map.items(): 
        rec[v] = rec.pop(k) # rename the key 
    
## Format HTML table 
column_alignments = {
    "Your Match Rank": "center",
    "Occupation Title": "left",
    "Average Wages (Annual)": "right",
    "Typical Education": "left",
    "Outlook": "left"
}

table = tabulate(
    rec_list,
    headers="keys",
    tablefmt="html",
    colalign=[column_alignments[col] for col in column_alignments.keys()]
)

## Styling the table 
table_headers = column_alignments.keys()
table_html = "<table>"
table_html += "<tr>"

# Header Style
header_style = "font-weight: bold; font-size: 20px;"
for header in table_headers:
    table_html += f"<th style='{header_style}'>{header}</th>"
table_html += "</tr>"

# Cell Style
cell_style = "font-weight: normal; font-size: 16px;"  
for rec in rec_list:
    table_html += "<tr>"
    for header in table_headers:
        table_html += f"<td style='{cell_style}'>{rec[header]}</td>"
    table_html += "</tr>"
table_html += "</table>"

NameError: name 'cos_recommendations' is not defined

In [12]:
message_style = "font-weight: bold; font-style: italic; font-size: 16px;"

with open("../funcs/email_message_text.txt", "r") as file: 
    message_text = file.read()

message_text
message = f'<div style="{message_style}">{message_text}:</div>\n\n{table_html}'


"Thank you for participating in our workforce survey about community member job skills, experiences, and interests. Your inputs will be extremely important as we seek to understand the employment strengths and desires of people from across the state, to make informed recommendations to workforce decision makers about how to improve access to employment for ALL Delawareans. \n\nAs promised, below are the results from the skills assessment portion of the survey. We have included recommendations of careers that you are well-suited for, based on your responses. You can click on each career to learn more! \n\nWe also wanted to share this informational flyer with important links to different workforce agencies and resources that may be useful to you in the future.\n\nFinally, we will be sending you your $10 gift card for participating in the coming days… look out for it in your inbox (and check your junk mail if you don't receive it in the next 10 days)! Please respond to this email if you h

In [ ]:
## TO-DO: Loop over the question_answer key and not the provided responses, avoid having to auto-fill again at the end

# processed_responses = []
# placeholder_processed_response_ids = []
# combined_map = combine_qa_keys()

# for resp in sm_survey_responses['data']:      
#     if resp['id'] not in placeholder_processed_response_ids:
#         # TO-DO: Do the email validation in the actual email sending function 
#         resp_dict = {
#         'response_id':resp['id'],
#         'collector_id':resp['collector_id'], 
#         'questions':[] 
#         }
#         resp_question_answers = {q['id']:q['answers'] for p in resp['pages'] for q in p['questions']}

#         ## Add questions information from combined qa key 
#         for q_map in  list(combined_map['non-skills-matcher'].values()) +  list(combined_map['skills-matcher'].values()):
#             # Fill-in omitted questions from key
#             if q_map['question_id']['sm'] not in resp_question_answers.keys():
#                 q_map['auto_filled'] = True
#                 # Auto-fill beginner level answer for skills-matcher questions
#                 q_map['answers'] = [q_map['answers'][0]] if q_map['question_type'] == 'skills-matcher' else None
#                 resp_dict['questions'].append(q_map)

#             elif q_map['answers'] is not None: # if the answer key has answer choices listed for the question
                
#                 q_map_answer_key = {a['id']['sm']:a for a in q_map['answers']}
#                 try: 
#                     resp_answers = [q_map_answer_key[a['choice_id']]
#                         if 'choice_id' in a.keys() else {'id':{'sm':a['other_id']}, 'text':{'sm':a['text']}}
#                         for a in resp_question_answers[q_map['question_id']['sm']]]
#                 except: 
#                     print(resp_question_answers[q_map['question_id']['sm']])
#                     print(q_map_answer_key)
#                 q_map['answers'] = resp_answers

#             else: 
#                 q_map['answers'] = resp_question_answers[q_map['question_id']['sm']]
                
#             resp_dict['questions'].append(q_map)

#     processed_responses.append(resp_dict)       

# processed_responses